<a href="https://colab.research.google.com/github/starship006/ARENA-work/blob/main/w3/scaling%20laws/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Write a script to train a small CNN on MNIST, or find one you have written previously."

Day 1 turned out to be more of a "relearn how to actually make a CNN, and then implement it out" type of day; I got a working CNN predicting on MNIST on 98% accuracy

In [ ]:
!pip install wandb

In [62]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm_notebook

import wandb

device = "cuda" if t.cuda.is_available() else "cpu"

In [56]:
class ConvNet(nn.Module):

    def __init__(self):
        super().__init__()
        
        out_one = 6
        out_two = 18
        kernel_size = 5

        lin_out_one = 120
        lin_out_two = 60
        lin_out_three = 10
        self.conv1 = nn.Conv2d(1, out_one, kernel_size)
        self.conv2 = nn.Conv2d(out_one, out_two, kernel_size)

        final_dim = (((28 - kernel_size + 1) / 2) - kernel_size + 1) / 2 # two comes from max pooling
        final_dim = int(final_dim) # convert float to int
        self.lin1 = nn.Linear(out_two * final_dim * final_dim, lin_out_one) 
        self.lin2 = nn.Linear(lin_out_one, lin_out_two)
        self.lin3 = nn.Linear(lin_out_two, lin_out_three)

    def forward(self, x):
        # apply convolutions
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        # flatten everything except the batch
        x = t.flatten(x, 1) 
        # apply linear layers
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x

In [47]:
mnist_trainset = datasets.MNIST(root = './data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = datasets.MNIST(root = './data', train=False, download=True, transform=transforms.ToTensor())

train_dl = DataLoader(mnist_trainset, batch_size=64, shuffle=True)
test_dl = DataLoader(mnist_testset, batch_size=64, shuffle=True)

In [48]:
mnist_trainset[0][0].shape

torch.Size([1, 28, 28])

In [49]:
def train_model(model, train_dl, test_dl, criterion, optimizer, num_epochs: int, val_history):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # train
        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0.0
            current_dl = None

            if phase == 'train':
                model.train()
                current_dl = train_dl
            else:
                model.eval()
                current_dl = test_dl
            

            for inputs, labels in current_dl:
                inputs = inputs.float().to(device)
                labels = labels.float().to(device)
                optimizer.zero_grad()

                with t.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    orig_labels = labels
                    labels = labels.to(t.int64)
                    labels = F.one_hot(labels, 10).to(t.float64)


                    loss = criterion(outputs, labels)
                    preds = t.argmax(outputs,dim=-1)

                    if (phase == 'train'):
                        loss.backward()
                        optimizer.step()
                    
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += t.sum(preds == orig_labels.data)

            epoch_loss = running_loss / len(current_dl.dataset) # TODO: divide by something else, right????
            epoch_acc = running_corrects / len(current_dl.dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val':
                val_history.append(epoch_acc)
        print() 
    return model, val_history

In [67]:
def train_with_wandb():
    wandb.init()
    # constant
    num_epochs = 3

    # wandb parameters
    lr = wandb.config.lr

    # set-up
    model = ConvNet().to(device)
    optim = t.optim.SGD(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss().to(device)

    examples_seen = 0
    wandb.watch(model, criterion = criterion, log="all", log_freq = 10, log_graph = True)

    mid_log_count = 20 # how many training steps to take before looking at test loss
    count = 0
    # train!
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0.0
            current_dl = None

            if phase == 'train':
                model.train()
                current_dl = train_dl
            else:
                model.eval()
                current_dl = test_dl
            

            for inputs, labels in current_dl:
                inputs = inputs.float().to(device)
                labels = labels.float().to(device)
                optim.zero_grad()

                with t.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    orig_labels = labels
                    labels = labels.to(t.int64)
                    labels = F.one_hot(labels, 10).to(t.float64)


                    loss = criterion(outputs, labels)
                    preds = t.argmax(outputs,dim=-1)

                    if (phase == 'train'):
                        loss.backward()
                        optim.step()
                        examples_seen += len(inputs)
                        # logging
                        running_loss += loss.item() * inputs.size(0)
                        running_corrects += t.sum(preds == orig_labels.data)
                        wandb.log({"train_loss" : loss}, step = examples_seen)

                        count += 1
                        if (count == mid_log_count):
                            

                    else:
                        running_loss += loss.item() * inputs.size(0)
                        running_corrects += t.sum(preds == orig_labels.data)
                        

            epoch_loss = running_loss / len(current_dl.dataset) # TODO: divide by something else, right????
            epoch_acc = running_corrects / len(current_dl.dataset)
            if (phase == 'val'):
                # logging
                wandb.log({"test_accuracy:" : epoch_acc}, step=examples_seen)
    filename = f"{wandb.run.dir}/model_state_dict.pt"
    print(f"Saving model to: {filename}")
    t.save(model.state_dict(), filename)
    wandb.save(filename)

In [68]:
sweep_config = {
    'method': 'random',
    'name': 'CNN_lr_sweep_0',
    'metric': {'name' : 'test_accuracy', 'goal' : 'maximize'},
    'parameters':
    {
        'lr': {'max': 0.1, 'min': 0.0001, 'distribution': 'log_uniform_values'}
    }
}

In [69]:
sweep_id = wandb.sweep(sweep = sweep_config, project='CNN_LR')
wandb.agent(sweep_id = sweep_id, function = train_with_wandb, count = 2)

Create sweep with ID: 91i499o9
Sweep URL: https://wandb.ai/starship006/CNN_LR/sweeps/91i499o9


wandb: Agent Starting Run: fpa74804 with config:
wandb: 	lr: 0.001061135472546422
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saving model to: /content/wandb/run-20221110_205959-fpa74804/files/model_state_dict.pt


test_accuracy:,▁▄█
train_loss,▃▅▅▆▁█▇▂▄▃▅▂▃▄▅▄▁▃▃▃▂▄▃▆▅▃▄▆▂▁▄▃▃▃▃▄▄▃▁▃
test_accuracy:,0.158
train_loss,2.29255


wandb: Agent Starting Run: qyqcj8sn with config:
wandb: 	lr: 0.05781639801214756
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saving model to: /content/wandb/run-20221110_210053-qyqcj8sn/files/model_state_dict.pt


test_accuracy:,▁▆█
train_loss,███▅▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
test_accuracy:,0.9798
train_loss,0.03668
